In [1]:
import time   # The rest of the content should be starting after the page is fully loaded.
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [80]:
link = 'https://www.amazon.in/'
print(link)

https://www.amazon.in/


## <font color='darkblue'>* Problems with direct link scraping *</font> 

In [81]:
res  = requests.get(link)
soup = BeautifulSoup(res.text, 'html.parser')
print(res)

<Response [200]>


In [82]:
soup.find_all('a')[4]

# Here other than the about and less important info all the other links are hidden when we try to scrap them instead the link
# is returning huge length of numbers.

<a aria-label="Choose a language for shopping." class="nav-a nav-a-2 icp-link-style-2" href="/customer-preferences/edit?ie=UTF8&amp;preferencesReturnUrl=%2F&amp;ref_=topnav_lang" id="icp-nav-flyout">
<span class="icp-nav-link-inner">
<span class="nav-line-1">
</span>
<span class="nav-line-2">
<span class="icp-nav-flag icp-nav-flag-in icp-nav-flag-lop"></span>
<div>EN</div>
<span class="nav-icon nav-arrow"></span>
</span>
</span>
</a>

*Output: 'AboutPressCopyrightContact usCreatorAdvertiseDevelopersTermsPrivacyPolicy & SafetyHow YouTube worksTest new features© 2023 Google LLCGeeksforGeeks - YouTube'*

## <font color='indianred'>STARTING WITH SELENIUM: Is a type of web driver used to make our life easy.</font>

In [83]:
# !pip install selenium
# !pip install chromedriver_binary
# !pip install webdriver.Chrome()

In [84]:
from selenium import webdriver  # For working with webdriver

# We can check the filepath of our chrome driver 
import chromedriver_binary

In [85]:
# Before using chrome webdriver we need to download that on our path.
# webdriver.Chrome()
chromedriver_binary.chromedriver_filename    # This will give me the path where the chrome driver is located.

'C:\\Users\\LENOVO\\anaconda3\\lib\\site-packages\\chromedriver_binary\\chromedriver.exe'

In [86]:
# If the webdriver.Chrome() function is not working then generate the path through the above function and then pass it in the 
# args of the webdriver.chrome function.

In [87]:
# !pip install webdriver-manager

In [218]:
# This solution was created with the help of stackoverflow link:
# https://stackoverflow.com/questions/60296873/sessionnotcreatedexception-message-session-not-created-this-version-of-chrome

from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10268\452801513.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## <font color='darkblue'>* Youtube link web scraping with selenium - I [Understanding the tags]*</font> 

In [219]:
browser = driver

In [123]:
link = 'https://www.youtube.com/c/GeeksforGeeksVideos/videos'
browser.get(link)

In [91]:
soup  = BeautifulSoup(browser.page_source, 'html.parser')

In [92]:
print(soup.text.replace('\n\n', ''))

•NaN / NaN 
 
      Back
      Search
        Search
      
GeeksforGeeks - YouTube


In [93]:
sp = soup.find('ytd-rich-grid-renderer')  # grid renderer is for the grids inside the soup there are 16 grids with each grid
                                # having some specific number of items

In [94]:
len(sp)

TypeError: object of type 'NoneType' has no len()

In [ ]:
len(sp.find_all('ytd-rich-item-renderer'))   # Items per row will be 3 and rows per scroll are 20. item renderer will give item
                                             # grid row will give the row

In [ ]:
sp.find_all('ytd-rich-item-renderer')[0].text  # This should give us data of the first video : It is giving the data.

In [ ]:
sp.find_all('ytd-rich-item-renderer')[1].text.replace('\n\n', '')

## <font color='darkblue'>* Youtube link web scraping with selenium - II [Data from Channel page]*</font> 

In [95]:
soup = soup.find_all('ytd-rich-item-renderer')

In [96]:
len(soup)

0

In [220]:
soup = BeautifulSoup(browser.page_source, 'html.parser')


In [146]:
data = []
for sp in soup.find_all('ytd-rich-item-renderer'):
    title           = sp.find('a', class_="yt-simple-endpoint focus-on-expand style-scope ytd-rich-grid-media" ).text

        # Fetch the href from the video
    video_link      = sp.find('a', class_="yt-simple-endpoint focus-on-expand style-scope ytd-rich-grid-media" ).get('href')

    try:
        views           = sp.find_all('span', class_='inline-metadata-item style-scope ytd-video-meta-block')[0].text
    except:
        views           = np.nan
    try:
        date_time       = sp.find_all('span', class_='inline-metadata-item style-scope ytd-video-meta-block')[1].text
    except:
        date_time       =  np.nan  

    try:
        thumbnail_link  = sp.find('img').get('src').split('?')[0]
    except:
        thumbnail_link  = np.nan
        
    data.append([title, views, date_time, video_link, thumbnail_link])

In [147]:
df = pd.DataFrame(data, columns=['title','views', 'date_time', 'video_link', 'thumbnail_link'])


In [148]:
df

,title,views,date_time,video_link,thumbnail_link
0,How to Use GPT 4 | How it's better from GPT 3 ...,1.4K views,17 hours ago,/watch?v=Mnx3TAgbX7o,https://i.ytimg.com/vi/Mnx3TAgbX7o/hqdefault.jpg
1,Dealing with Employment Cutbacks with Saheb Ku...,584 views,4 days ago,/watch?v=cYoeQbOXZJo,https://i.ytimg.com/vi/cYoeQbOXZJo/hqdefault.jpg
2,Master DSA Today and be the Masters of Tomorrow,17K views,5 days ago,/watch?v=PsOXP5hg52w,https://i.ytimg.com/vi/PsOXP5hg52w/hqdefault.jpg
3,How to avoid getting Laid Off | ft. Dr. Sneha ...,788 views,6 days ago,/watch?v=s4frRcj17AM,https://i.ytimg.com/vi/s4frRcj17AM/hqdefault.jpg
4,Psychology of Deciding Your Career ft. Dhairya...,688 views,7 days ago,/watch?v=0oPyv39ynTo,https://i.ytimg.com/vi/0oPyv39ynTo/hqdefault.jpg
...,...,...,...,...,...
1637,Length of shortest chain to reach a target wor...,40K views,6 years ago,/watch?v=6pIC20wCm20,https://i.ytimg.com/vi/6pIC20wCm20/hqdefault.jpg
1638,Binary Search | GeeksQuiz,187K views,6 years ago,/watch?v=T2sFYY-fT5o,https://i.ytimg.com/vi/T2sFYY-fT5o/hqdefault.jpg
1639,Number of Triangles in an Undirected Graph | G...,17K views,7 years ago,/watch?v=ChdNz1Ui1uc,https://i.ytimg.com/vi/ChdNz1Ui1uc/hqdefault.jpg
1640,Write a program to print all permutations of a...,473K views,7 years ago,/watch?v=AfxHGNRtFac,https://i.ytimg.com/vi/AfxHGNRtFac/hqdefault.jpg


In [221]:
df.isnull().sum()

title               0
views               0
date_time           0
video_link          0
thumbnail_link    108
dtype: int64

In [150]:
df.to_csv('youtube-data.csv', index=False)

In [259]:
browser = webdriver.Chrome(ChromeDriverManager().install())

browser.get('https://www.youtube.com')
time.sleep(3)

data = []

for link in tqdm(df['video_link']):
    link = 'https://www.youtube.com'+ link
    browser.get(link)
    
    time.sleep(5) # Once the page is loaded after that we will scrap the data.
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    # Title of the video scraping
    try:
        title = soup.find('h1', class_ = 'style-scope ytd-watch-metadata').text.strip()
    except:
        title = np.nan
    
    # how many views
    try:
        view = soup.find_all('span', class_ = 'style-scope yt-formatted-string bold')[0].text
    except:
        view = np.nan
        
    
    # Upload date
    try:
        date_time = soup.find_all('span', class_ = 'style-scope yt-formatted-string bold')[2].text
    except:
        date_time = np.nan
    
    # Scrap the likes in a video
    try:
        like = soup.find('button', class_="yt-spec-button-shape-next yt-spec-button-shape-next--tonal yt-spec-button-shape-next--mono yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--segmented-start").get('aria-label').split(' ')[-3]
    except:
        like = np.nan
    
    # Scraping the description
    try:
        description = soup.find('span', class_ = 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap').text
    except:
        description = np.nan
        
    data.append([title, date_time, view, like, link, description])
    
    

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10268\891596756.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())
100%|████████████████████████████████████████████████████████████████████████████| 1642/1642 [3:25:18<00:00,  7.50s/it]


In [243]:
# Demostration of time library

for i in range(1,10):
    print(i)
    time.sleep(1)

1
2
3
4
5
6
7
8
9


In [261]:
df = pd.DataFrame(data, columns = ['title', 'date_time', 'view', 'like', 'link', 'description'])

In [264]:
df.to_csv('youtube-gfg-data.csv', index = False)

In [265]:
df.isnull().sum()

title          0
date_time      0
view           0
like           0
link           0
description    0
dtype: int64

In [268]:
# Changing the video-link to complete names

print(df['link'][0])

https://www.youtube.com/watch?v=Mnx3TAgbX7o


In [266]:
df.head()

,title,date_time,view,like,link,description
0,How to Use GPT 4 | How it's better from GPT 3 ...,1 day ago,2K views,70,https://www.youtube.com/watch?v=Mnx3TAgbX7o,"OpenAI released GPT4 on March 14, and it's way..."
1,Dealing with Employment Cutbacks with Saheb Ku...,5 days ago,586 views,17,https://www.youtube.com/watch?v=cYoeQbOXZJo,Saheb Kumar who is an SDE intern at Amazon con...
2,Master DSA Today and be the Masters of Tomorrow,6 days ago,18K views,77,https://www.youtube.com/watch?v=PsOXP5hg52w,Are you ready to level up your coding game and...
3,How to avoid getting Laid Off | ft. Dr. Sneha ...,7 days ago,794 views,27,https://www.youtube.com/watch?v=s4frRcj17AM,Dr. Sneha Sharma a Career Coach and a Public S...
4,Psychology of Deciding Your Career ft. Dhairya...,8 days ago,689 views,30,https://www.youtube.com/watch?v=0oPyv39ynTo,"Meet Dhairya, who wears the hats of a Tedx Spe..."


In [2]:
df = pd.read_csv('youtube-gfg-data.csv')

In [3]:
df.head()

,title,date_time,view,like,link,description
0,How to Use GPT 4 | How it's better from GPT 3 ...,1 day ago,2K views,70,https://www.youtube.com/watch?v=Mnx3TAgbX7o,"OpenAI released GPT4 on March 14, and it's way..."
1,Dealing with Employment Cutbacks with Saheb Ku...,5 days ago,586 views,17,https://www.youtube.com/watch?v=cYoeQbOXZJo,Saheb Kumar who is an SDE intern at Amazon con...
2,Master DSA Today and be the Masters of Tomorrow,6 days ago,18K views,77,https://www.youtube.com/watch?v=PsOXP5hg52w,Are you ready to level up your coding game and...
3,How to avoid getting Laid Off | ft. Dr. Sneha ...,7 days ago,794 views,27,https://www.youtube.com/watch?v=s4frRcj17AM,Dr. Sneha Sharma a Career Coach and a Public S...
4,Psychology of Deciding Your Career ft. Dhairya...,8 days ago,689 views,30,https://www.youtube.com/watch?v=0oPyv39ynTo,"Meet Dhairya, who wears the hats of a Tedx Spe..."
